# ML_Project

1) 데이터를 구체적으로 살펴 보기에 앞서 간단한 전처리 이 후 modeling을 통해 성능이 어떻게 나오는지 살펴보자. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### train data set의 row가 400만개 이상이므로 Kaggle에서 제공하는 코드를 통해 data의 memory usage를 줄인 다음 modeling 작업을 진행한다. (그렇지 않는다면 중간에 kernel이 끊기는 문제가 발생할 수 있다.)

## Memory Reducing

In [2]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [3]:
data = reduce_mem_usage(pd.read_csv('./data/train_V2.csv'))

Memory usage of dataframe is 983.90 MB
Memory usage after optimization is: 288.39 MB
Decreased by 70.7%


In [ ]:
pd.set_option('display.max_columns', 30)
data.head(2)

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.0,0,0,0,60,1241,0,0,0.0,1306,squad-fpp,28,26,-1,0,0.000000,0,0.000000,0,0,244.75,1,1466,0.444336
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.5,0,0,0,57,0,0,0,0.0,1777,squad-fpp,26,25,1484,0,0.004501,0,11.039062,0,0,1434.00,5,0,0.640137


In [4]:
data_2 = data.copy()
data_3 = data.copy()

In [5]:
## rough하게 object column 모두 제외하고 모델링
## rankpoint, maxPlace, roadKills, matchDuration, teamKills

data = data.drop(columns=['Id', 'groupId', 'matchId', 'rankPoints', 'roadKills', 'matchDuration', 'teamKills'])

data.isna().sum()

assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchType          0
maxPlace           0
numGroups          0
revives            0
rideDistance       0
swimDistance       0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

In [ ]:
data_2 = data_2.drop(columns=['Id', 'groupId', 'matchId', 'rankPoints', 'roadKills', 'matchDuration', 'teamKills', 'killPoints', 'winPoints', 'maxPlace'])

data.isna().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

In [ ]:
data = data.dropna()

In [ ]:


# matchType2라는 칼럼을 생성
data_2['matchType2'] = pd.NaT

# row가 많아서 생기는 메모리 에러 무시하고 진행
# pd.set_option('mode.chainedassignment',  None)

## crash와 flare를 event mode로 처리할 때    채택
maplist = {
    'squad' : 'squad',
    'duo' : 'duo',
    'solo' : 'solo',
    'crash' : 'event',
    'flare' : 'event'
}

for i in maplist:
  data_2.matchType2[(data_2['matchType'].str.contains(i)==True)&(data_2['matchType2'].isnull())]=maplist.get(i)


# solo ⮕ 1, duo ⮕ 2, squad ⮕ 4, falre & crash ⮕ 0 으로 전환
def convert(x):
    if x == "squad":
        return 4
    elif x == "duo":
        return 2
    elif x == "solo":
        return 1
    else : 
        return 0

data_2['matchType2'] = data_2['matchType2'].map(convert)


# 기존의 matchType drop
data_2.drop(['matchType'], axis=1, inplace=True) 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data.head(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,maxPlace,numGroups,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,matchType2
0,0,0,0.0,0,0,0,60,1241,0,0,0.0,28,26,0,0.000000,0.000000,0,244.75,1,1466,0.444336,4
1,0,0,91.5,0,0,0,57,0,0,0,0.0,26,25,0,0.004501,11.039062,0,1434.00,5,0,0.640137,4


In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns='winPlacePerc')
y = data['winPlacePerc']

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=100)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3557572, 21), (889393, 21), (3557572,), (889393,))

In [ ]:
X = data_2.drop(columns='winPlacePerc')
y = data_2['winPlacePerc']

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=100)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3557572, 18), (889394, 18), (3557572,), (889394,))

## Installing CatBoost

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 86 kB/s 


## Modeling

    - LinearRegression
    - Lasso
    - Ridge
    - SVR
    - KNeighborsRegressor
    - DecisionTreeRegressor
    - LGBMRegressor
    - RandomForestRegressor
    - XGBRegressor
    - CatBoostRegressor

In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge

# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor

from lightgbm.sklearn import LGBMRegressor
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error

## Linear Reg, Lasso, Ridge, LGBMRegressor, RandomFor, XGBReg, CatBoost

### lightGBM

In [ ]:
model = LGBMRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
mean_absolute_error(y_test, pred)

0.06330111985584375

In [ ]:
model = LGBMRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
mean_absolute_error(y_test, pred)

0.473042242613814